In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
ss = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')
data = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
#I could create a func and pass each individual column into the func but, I don't wanna. Lets automate this shizz

In [ ]:
data.cat0.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
def preprocess_col(col):
    le.fit(col)
    return le.fit_transform(col)

In [ ]:
data.cat0.value_counts()

In [ ]:
def cols_to_change(data):
    data.cat0 = preprocess_col(data.cat0)
    data.cat1 = preprocess_col(data.cat1)
    data.cat2 = preprocess_col(data.cat2)
    data.cat3 = preprocess_col(data.cat3)
    data.cat4 = preprocess_col(data.cat4)
    data.cat5 = preprocess_col(data.cat5)
    data.cat6 = preprocess_col(data.cat6)
    data.cat7 = preprocess_col(data.cat7)
    data.cat8 = preprocess_col(data.cat8)
    data.cat9 = preprocess_col(data.cat9)
    data.cat10 = preprocess_col(data.cat10)
    data.cat11 = preprocess_col(data.cat11)
    data.cat12 = preprocess_col(data.cat12)
    data.cat13 = preprocess_col(data.cat13)
    data.cat14 = preprocess_col(data.cat14)
    data.cat15 = preprocess_col(data.cat15)
    data.cat16 = preprocess_col(data.cat16)
    data.cat17 = preprocess_col(data.cat17)
    data.cat18 = preprocess_col(data.cat18)
    return data

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
target = data.pop('target')
test.drop('id',1,inplace=True)
test_data = cols_to_change(test)

In [ ]:
data.drop('id',1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics  import accuracy_score

In [ ]:
data[['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18']][:30]

In [ ]:
data = cols_to_change(data)

In [ ]:
all_cond = data['cont0'] + data['cont1'] + data['cont2'] +  data['cont3'] + data['cont4'] + data['cont5'] +  data['cont6'] + data['cont7'] + data['cont8'] + data['cont9'] + data['cont10']

In [ ]:
all_cond_mean = all_cond / 10 

In [ ]:
data['cond_mean'] = all_cond_mean.values

In [ ]:
data[:1]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data,target,test_size=0.2)

In [ ]:
import lightgbm

In [ ]:
light_model = lightgbm.LGBMModel(boosting_type='gbdt', num_leaves=31, max_depth=50,
                 learning_rate=0.005, n_estimators=3000,
                 subsample_for_bin=300000, objective='binary', class_weight=None,
                 min_split_gain=0.1, min_child_weight=1e-8, min_child_samples=15,
                 subsample=1., subsample_freq=0, colsample_bytree=1.,
                 reg_alpha=0., reg_lambda=0., random_state=None,
                 n_jobs=-1, silent=False, importance_type='split')

In [ ]:
%%time
light_model.fit(x_train,y_train)

In [ ]:
y_preds = light_model.predict(x_test)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test,y_preds)

In [ ]:
test_all_cond = test['cont0'] + test['cont1'] + test['cont2'] +  test['cont3'] + test['cont4'] + test['cont5'] +  test['cont6'] + test['cont7'] + test['cont8'] + test['cont9'] + test['cont10']
test_all_cond_mean = test_all_cond / 10
test['cond_mean'] = test_all_cond_mean

In [ ]:
preds = light_model.predict(test)
ss['target'] = preds
ss.to_csv('light_model.csv',index=False)
ss